In [1]:
sc

In [19]:
ls = sc.textFile('data/coupon150720.csv', 100) #indico con 100 el número de particiones donde almacenar los datos

In [21]:
ls.getNumPartitions()

100

In [5]:
ls.take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [20]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [13]:
ls.count()

1232662

In [16]:
ls.cache()

MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:-2

In [18]:
ls.count()

1232662

In [17]:
#Controlar la cache de los datos
ls.persist()

1232662

¿Cuántos pasajeros han volado con Iberia?

In [39]:
ls.map(lambda x: x.split(',')).take(5)

[[u'79062005698500',
  u'1',
  u'MAA',
  u'AUH',
  u'9W',
  u'9W',
  u'56.79',
  u'USD',
  u'1',
  u'H',
  u'H',
  u'0526',
  u'150904',
  u'OK',
  u'IAF0'],
 [u'79062005698500',
  u'2',
  u'AUH',
  u'CDG',
  u'9W',
  u'9W',
  u'84.34',
  u'USD',
  u'1',
  u'H',
  u'H',
  u'6120',
  u'150905',
  u'OK',
  u'IAF0'],
 [u'79062005924069',
  u'1',
  u'CJB',
  u'MAA',
  u'9W',
  u'9W',
  u'60.0',
  u'USD',
  u'1',
  u'H',
  u'H',
  u'2768',
  u'150721',
  u'OK',
  u'IAA0'],
 [u'79065668570385',
  u'1',
  u'DEL',
  u'DXB',
  u'9W',
  u'9W',
  u'160.63',
  u'USD',
  u'2',
  u'S',
  u'S',
  u'0546',
  u'150804',
  u'OK',
  u'INA0'],
 [u'79065668737021',
  u'1',
  u'AUH',
  u'IXE',
  u'9W',
  u'9W',
  u'152.46',
  u'USD',
  u'1',
  u'V',
  u'V',
  u'0501',
  u'150803',
  u'OK',
  u'INA0']]

In [41]:
#mi solución inicial
aerolineas = ls.map(lambda x: x.split(',')).filter(lambda x: x[5]=='IB').count()

26158

In [42]:
# solución más óptima de rendimiento
aerolineas = ls.map(lambda x: x.split(',')[5]).filter(lambda x: x=='IB').count()

In [43]:
aerolineas

26158

In [44]:
def getAerolineaDinero(l):
    elems = l.split(",")
    aerolinea = elems[5]
    dinero = float(elems[6])
    return (aerolinea, dinero)


In [45]:
getAerolineaDinero(ls.first())

(u'9W', 56.79)

In [46]:
dineros = ls.map(getAerolineaDinero)

In [48]:
dineros.take(3)

[(u'9W', 56.79), (u'9W', 84.34), (u'9W', 60.0)]

In [53]:
dineros.filter(lambda x: x[0]=='IB').take(3)

[(u'IB', 313.99), (u'IB', 865.32), (u'IB', 16.58)]

In [54]:
dinerosIB = dineros.filter(lambda x: x[0]=='IB')

In [64]:
cantidades = dinerosIB.map(lambda x: x[1])

In [67]:
cantidades.take(3)

[313.99, 865.32, 16.58]

In [66]:
cantidades.sum()

2828044.420000028

In [65]:
cantidades.stats()

(count: 26158, mean: 108.113939139, stdev: 168.011485542, max: 5239.93, min: 0.0)

        #### ¿Cuál es la ruta con el billete más caro?

In [68]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [72]:
def getRutaDinero(l):
    e = l.split(',')
    origen = e[2]
    destino = e[3]
    dinero = float(e[6])
    ruta = origen + '-' + destino
    return (ruta, dinero)

In [73]:
getRutaDinero(ls.first())

(u'MAA-AUH', 56.79)

In [74]:
rutas = ls.map(getRutaDinero)

In [75]:
rutas.take(3)

[(u'MAA-AUH', 56.79), (u'AUH-CDG', 84.34), (u'CJB-MAA', 60.0)]

In [77]:
#ordenamos desc
rutas.sortBy(lambda x: x[1], False).take(5)

[(u'SYD-CGK', 6355194.0),
 (u'SYD-CGK', 6355194.0),
 (u'SYD-CGK', 6355194.0),
 (u'CDG-IAD', 637313.52),
 (u'IAD-CDG', 637313.52)]

In [78]:
# como hay importes incorrectos (6355194 un billete) vamos a filtrar
rutas.filter(lambda x: x[1] < 7500).sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'MIA-CCS', 7415.0)]

In [79]:
filtrado = rutas.filter(lambda x: x[1] < 7500)

In [81]:
filtrado.cache()

PythonRDD[87] at RDD at PythonRDD.scala:43

In [82]:
filtrado.groupByKey().take(3)

[(u'TRV-MCT', <pyspark.resultiterable.ResultIterable at 0x7f6dd3282090>),
 (u'IAD-SFO', <pyspark.resultiterable.ResultIterable at 0x7f6dd32826d0>),
 (u'YVR-TPE', <pyspark.resultiterable.ResultIterable at 0x7f6dd3282490>)]

In [84]:
#precio máximo por ruta
filtrado.groupByKey().mapValues(lambda vs: max(vs)).take(3)

[(u'TRV-MCT', 220.44), (u'IAD-SFO', 1578.63), (u'YVR-TPE', 877.71)]

In [85]:
maximos = filtrado.groupByKey().mapValues(lambda vs: max(vs))

In [86]:
maximos.sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'CCS-MIA', 7415.0),
 (u'MIA-CCS', 7415.0),
 (u'RUH-LAX', 7401.94),
 (u'DXB-MAA', 7343.27)]

#### Ruta con el valor medio más alto

In [89]:
medios = filtrado.groupByKey().mapValues(lambda vs: sum(vs) / len(vs))

In [88]:
medios.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 77.0 failed 1 times, most recent failure: Lost task 0.0 in stage 77.0 (TID 2994, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1295, in takeUpToNumLeft
    yield next(iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 1907, in <lambda>
    map_values_fn = lambda kv: (kv[0], f(kv[1]))
  File "<ipython-inpu